# Phase 4 Project: Skin Condition Classification
## Technical Analysis Notebook
### Authored by Trinity Gahagen
---
---

This notebook and the modeling notebook both work with the Skin Condition image dataset found on this [Kaggle page](https://www.kaggle.com/datasets/syedalinaqvi/augmented-skin-conditions-image-dataset).